## Imports

In [1]:
import os
import cv2
import pickle
import imageio
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('Agg')
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm
from pprint import pprint

In [2]:
from enum import Enum
from collections import namedtuple

class ModelType(Enum):
    CLASSIFICATION = 0
    SEGMENTATION = 1

classlabels_viz_colors = ['black', 'green', 'yellow', 'blue', 'red', 'magenta', 'cyan',
                          'lightseagreen', 'brown', 'magenta', 'olive', 'wheat', 'white', 'black']
classlabels_viz_bounds = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 100]

classlabels_viz_cmap = mpl.colors.ListedColormap(classlabels_viz_colors)
classlabels_viz_norm = mpl.colors.BoundaryNorm(classlabels_viz_bounds, classlabels_viz_cmap.N)

confidence_heatmap_viz_colors = ['black', 'blue', 'red', 'orange', 'yellow', 'lightgreen', 'lightseagreen']
confidence_heatmap_viz_bounds = [-1, 0,0.5,0.6,0.7,0.8,0.9,1]
confidence_heatmap_viz_cmap = mpl.colors.ListedColormap(confidence_heatmap_viz_colors)
confidence_heatmap_viz_norm = mpl.colors.BoundaryNorm(confidence_heatmap_viz_bounds, confidence_heatmap_viz_cmap.N)


LabelColor = namedtuple('LabelColor', ['name', 'id', 'trainid', 'color', 'category'])

LABEL_COLORS = [
    LabelColor('class1', 1, 0, (128, 0, 128), 'driveableterrain'),
    LabelColor('class2', 2, 1, (255, 0, 0), 'non-driveableterrain'),
    LabelColor('class3', 3, 2, (0, 0, 255), 'sky'),
    LabelColor('class4', 4, 3, (0, 255, 0), 'trees'),
    LabelColor('class5', 5, 4, (255, 0, 255), 'implement'),
    LabelColor('class6', 6, 5, (255, 255, 0), 'basket markers')
]

LABEL_COLORS_4CLASS = LABEL_COLORS[0:4]
LABEL_COLORS_5CLASS = LABEL_COLORS[0:5]
LABEL_COLORS_6CLASS = LABEL_COLORS[0:6]
LABEL_COLORS_SKY_DET = [LABEL_COLORS[0], LABEL_COLORS[2]]

LABEL_COLORS_IMPL = [
    LabelColor('class1', 1, 1, (128, 0, 128), 'implement'),
    LabelColor('class2', 2, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 3, 3, (0, 0, 255), 'harrow_tine'),
    LabelColor('class4', 4, 4, (0, 255, 0), 'basket'),
    LabelColor('class5', 5, 5, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

LABEL_COLORS_IMPL_REDUCED = [
    LabelColor('class0', 1, 0, (0, 0, 0), 'background'),
    LabelColor('class1', 2, 1, (0, 255, 0), 'implement'),
    LabelColor('class2', 3, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 4, 3, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

PLUG_LABEL_MAP ={0: 'no-plug', 1: 'plug'}
IMPL_SEGMENT_LABEL_MAP = {0: 'background', 1: 'implement', 2: 'sweep', 3:'basket_marker'}

In [3]:
def create_mpl_viz_outputs(output_path,
                           image,
                           prediction_labels,
                           confidences,
                           depth_img, 
                           image_title='Image', 
                           pred_title='Prediction', 
                           conf_title='Confidence', 
                           depth_title='Depth',
                           bbox_coords=[]):
    """
    Utility function to plot results based on order of input provided.

    Axes index can have different values based on input provided.

    For example: If image, prediction and groundtruth_label is provided, A three axes plot will be generated with
    image(ax1), ground_truth(ax2), prediction(ax3).

    Order of plot if all the inputs are provided will be in same order as arguments listed above.
    """
    axis_index = list(range(len(list(filter(lambda x: x is not None, [image,
                                                                      prediction_labels, confidences,
                                                                      depth_img])))))
    axis_curr_index = 0
    fig, axes = mpl.pyplot.subplots(1, len(axis_index), figsize=((60, 30)))
    if bbox_coords:
#         xmin, ymin, xmax, ymax = bbox_coords
        xmin, xmax, ymin, ymax = bbox_coords
        if (xmin < 0) or (ymin < 0):
            raise ValueError(f'Either {xmin} or {ymin} are negative')
        else:
            rect = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
                                     facecolor='none')
    else:
        rect = None

    if axis_curr_index < len(axes):
        # Grab only RGB channels from image, otherwise depth with distort the image when it is displayed
        axes[axis_curr_index].imshow(image)
        axes[axis_curr_index].set_title(image_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(depth_img, cmap='turbo')
        axes[axis_curr_index].set_title(depth_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(prediction_labels, classlabels_viz_cmap, classlabels_viz_norm, interpolation='nearest')
        if rect is not None:
            rect1 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',facecolor='none')
            axes[axis_curr_index].add_patch(rect1)
        axes[axis_curr_index].set_title(pred_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        c = np.max(confidences, axis=2)
        axes[axis_curr_index].imshow(c, confidence_heatmap_viz_cmap, confidence_heatmap_viz_norm, interpolation='nearest')
#         if rect is not None:
#             rect2 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
#                                           facecolor='none')
#             axes[axis_curr_index].add_patch(rect2)
        axes[axis_curr_index].set_title(conf_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    mpl.pyplot.savefig(output_path, pad_inches=0, bbox_inches='tight', dpi=150)
    mpl.pyplot.close('all')

def read_image(image_path):
    image = (np.load(image_path) * 255).astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image

def normalize_and_clip_depth(depth, max_depth):
    """
    Return an optionally normalized (and clipped) depth.
    """
    depth[np.isnan(depth)] = max_depth
    depth[depth > max_depth] = max_depth
    depth = ((depth) / max_depth).astype(np.float32)
    return depth

DEFAULT_TONEMAP_PARAMS = {"policy": "tonemap", "alpha": 0.25, "beta": 0.9, "gamma": 0.9, "eps": 1e-6}
def normalize_image(image, hdr_mode=True, normalization_params=DEFAULT_TONEMAP_PARAMS, return_8_bit=False):
    """
    Normalize an 8 bit image according to the specified policy.
    If return_8_bit, this returns an np.uint8 image, otherwise it returns a floating point
    image with values in [0, 1].
    """
    normalization_policy = normalization_params['policy']
    lower_bound = 0
    upper_bound = 1
    if np.isnan(hdr_mode):
        hdr_mode = False

    if hdr_mode and image.dtype == np.uint8:
        # The image was normalized during pack-perception (tonemap)
        if return_8_bit:
            return image
        lower_bound = 0.0
        upper_bound = 255.0
    elif normalization_policy == "percentile" and hdr_mode:
        lower_bound = np.array([np.percentile(image[..., i],
                                              normalization_params['lower_bound'],
                                              interpolation='lower')
                                for i in range(3)])
        upper_bound = np.array([np.percentile(image[..., i],
                                              normalization_params['upper_bound'],
                                              interpolation='lower')
                                for i in range(3)])
    elif normalization_policy == "percentile_vpu" and hdr_mode:
        r, g, b = image[..., 0], image[..., 1], image[..., 2]
        brightness = (3 * r + b + 4 * g) / 8
        lower_bound = np.percentile(brightness, normalization_params['lower_bound'],
                                    interpolation='lower')
        upper_bound = np.percentile(brightness, normalization_params['upper_bound'],
                                    interpolation='lower')
    elif normalization_policy == "3sigma" and hdr_mode:
        sigma_size = normalization_params['sigma_size']
        min_variance = normalization_params['min_variance']
        r, g, b = image[..., 0], image[..., 1], image[..., 2]
        brightness = (3 * r + b + 4 * g) / 8
        mean, sigma = np.mean(brightness), np.std(brightness)
        brightness_min, brightness_max = np.min(brightness), np.max(brightness)
        if (sigma * sigma_size) > mean:
            lmin = brightness_min
            lmax = min(brightness_max, mean * sigma_size)
            if (lmax - lmin) < min_variance:
                lmax = lmin + min_variance
            lower_bound = lmin
            upper_bound = lmax
        else:
            mean_var = mean - sigma_size * sigma
            output_min = max(brightness_min, mean_var)
            mean_var = mean + sigma_size * sigma
            output_max = min(brightness_max, mean_var)
            if (output_max - output_min) < min_variance:
                output_min = mean - min_variance / 2.0
                output_min = 0 if output_min < 0 else output_min
                output_max = output_min + min_variance
            lower_bound = output_min
            upper_bound = output_max
    elif normalization_policy == 'tonemap' and hdr_mode:
        if image.dtype != np.float32 and image.dtype != np.uint32:
            raise ValueError('HDR image type is {} instead of float32 or uint32'.format(image.dtype))
        alpha = normalization_params.get('alpha', DEFAULT_TONEMAP_PARAMS['alpha'])
        beta = normalization_params.get('beta', DEFAULT_TONEMAP_PARAMS['beta'])
        gamma = normalization_params.get('gamma', DEFAULT_TONEMAP_PARAMS['gamma'])
        eps = normalization_params.get('eps', DEFAULT_TONEMAP_PARAMS['eps'])

        r, g, b = image[..., 0], image[..., 1], image[..., 2]
        lum_in = 0.2126 * r + 0.7152 * g + 0.0722 * b
        lum_norm = np.exp(gamma * np.mean(np.log(lum_in + eps)))
        c = alpha * lum_in / lum_norm
        c_max = beta * np.max(c)
        lum_out = c / (1 + c) * (1 + c / (c_max ** 2))
        image = image * (lum_out / (lum_in + eps))[..., None]
    elif normalization_policy == "none" and hdr_mode:
        lower_bound = 0.0
        upper_bound = 2**20 - 1
    elif normalization_policy == "default" or not hdr_mode:
        assert np.max(image) <= 255 and np.min(image) >= 0, "Image with default " \
            "mode should be in range [0,255]"
        lower_bound = 0.0
        upper_bound = 255.0
    else:
        raise ValueError(
            f"--normalization-policy '{normalization_policy}' is not supported! "
            f"(on image with hdr_mode={hdr_mode})")

    image = (image.astype(np.float32, copy=False) - lower_bound) / (upper_bound - lower_bound)

    if return_8_bit:
        image = np.clip(image * 255.0, 0.0, 255.0)
        image = np.uint8(image)
    else:
        image = np.clip(image, 0.0, 1.0)

    return image

def read_saved_frame(pred_dir, image_id):
    states_to_save = ['', 'false_positive', 'false_negative', 'large_object_false_negative', 'true_positive', 'true_negative']
    frame = None
    for state in states_to_save:
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.png')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.png'))
            break
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.jpg')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.jpg'))
            break
    return frame

def read_images(pred_dir, _id):
    if not os.path.isfile(os.path.join(pred_dir, _id+'_image.npy')):
        return None, None, None, None
    image = np.load(os.path.join(pred_dir, _id+'_image.npy'))
    
    # 100m capped depth
    depth = np.load(os.path.join(pred_dir, _id+'_depth.npy'))
#     # raw depth
#     raw_depth_dir = '/raum_raid/li.yu/data/Jupiter_rock_demo_2021/Jupiter_rock_demo_loamy06_Oct20_2021/model_processed_v4.1_sky_2e-3_lr_1e-3_color_aug_full_model_LR_consistency_regularization_0.2_epoch_23/images/'
#     stereo_data = np.load(os.path.join(raw_depth_dir, _id, 'stereo_output.npz'))
#     depth = stereo_data['point_cloud'][:,:,-1]
#     depth = normalize_and_clip_depth(depth, 200)
    
    pred_label = np.load(os.path.join(pred_dir, _id+'_pred_label.npy'))
    confidence = np.load(os.path.join(pred_dir, _id+'_confidence.npy'))
    return image, depth, pred_label, confidence

# def create_frame(pred_dir, pred_merged_dir, _id, recreate=False):
#     canvas_path = os.path.join(pred_merged_dir, _id+'.png')
#     if recreate:
#         image, depth, pred_label, confidence = read_images(pred_dir, _id)
#         if image is None:
#             return None
#         create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth)
#     frame = cv2.imread(canvas_path)
#     return frame

def get_bbox_coords(i=-1, bbox_range_list=[], bbox_coord_list=[]):
    for bi in range(len(bbox_range_list)):
        bbox_range = bbox_range_list[bi]
        if bbox_range[0] <= i <= bbox_range[1]:
            return bbox_coord_list[bi]
    return []

def process_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
    return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    canvas_path = os.path.join(pred_merged_dir, _id+'.png')
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return None, None, None, None, None, None, None, None
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
        create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, image_title=image_title, bbox_coords=bbox_coords)
    frame = cv2.imread(canvas_path)
    return frame, image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_diff_frame(pred_merged_dir, _id, image1, depth1, pred_label1, confidence1, 
                      image2, depth2, pred_label2, confidence2, pred_title='prediction', conf_title='confidence'):
    canvas_path = os.path.join(pred_merged_dir, _id+'_diff.png')
    image = np.abs(image1 - image2)
    depth = np.abs(depth1 - depth2)
    pred_label = np.abs(pred_label1 - pred_label2)
    confidence = np.abs(confidence1 - confidence2)
    create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, conf_title=conf_title)
    frame = cv2.imread(canvas_path)
    return frame

def read_raw_image_by_id(data_dir, _id):
    image_path = os.path.join(data_dir, 'images', _id, 'artifact_debayeredrgb_0_'+_id+'.png')
    image = cv2.imread(image_path)
    return image

def read_raw_image_by_row(data_dir, row):
    image = cv2.imread(os.path.join(data_dir, row.artifact_debayeredrgb_0_save_path))
    return image

def create_video(ids, pred_dir, video_name, read_func=read_saved_frame, fps=2):
    frame = read_func(pred_dir, ids[10])
    height, width, layers = frame.shape
    print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width,height), isColor=True)
    
    good = 0
    for _id in tqdm(ids):
        frame = read_func(pred_dir, _id)
        if frame is not None:
            video.write(frame)
            good += 1
    print('total', len(ids), 'used', good)

    # cv2.destroyAllWindows()
    video.release()
    
from datetime import datetime, timedelta

def get_sequences(df, interval=5*60, per_camera=False):
    df = df.sort_values('collected_on')
    df['datetime'] = df.collected_on.apply(datetime.fromisoformat)
    sequence_dfs = []
    delta = timedelta(seconds=interval)
    start = True
    i0, i = 0, 0
    while i < len(df):
        if start:
            t0 = df.iloc[i].datetime
            start = False
        else:
            t1 = df.iloc[i].datetime
            if t1 - t0 > delta or i == len(df) - 1:
                chunk_df = df.iloc[i0 : i if i < len(df) - 1 else len(df)]
                if per_camera:
                    camera_locations = chunk_df.camera_location.unique()
                    camera_locations.sort()
                    for camera_location in camera_locations:
                        sequence_df = chunk_df[chunk_df.camera_location == camera_location]
                        sequence_df = sequence_df.sort_values('collected_on')
                        sequence_dfs.append(sequence_df)
                else:
                    sequence_dfs.append(chunk_df)
                start = True
                i0 = i
            else:
                t0 = t1
        i += 1
    return sequence_dfs

## Create video from raw left images

In [25]:
# create video from raw left images
# data_dir = '/data/jupiter/datasets/halo_failure_case_of_box_in_dust'
# data_dir = '/data/jupiter/datasets/halo_missed_lo_rock_0509_stereo'
# data_dir = '/data/jupiter/li.yu/data/halo_sample_terrains_15images'
# data_dir = '/data/jupiter/li.yu/data/halo_sample_terrains_15images_all_camera_sequence'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_day_collection_may29'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_night_collection_june03'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_night_collection_june03_2'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_day_collection_back_june05'
# data_dir = '/data/jupiter/datasets/halo_human_in_dust_dusk_collection_front_june07'
# data_dir = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_march2024'
# data_dir = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june06'
data_dir = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june07'
df = pd.read_csv(os.path.join(data_dir, 'annotations.csv'))
df = df.sort_values('collected_on')

# ids = df.id.to_list()
# video_name = os.path.join(data_dir, f'video.mp4')
# create_video(ids, data_dir, video_name, read_raw_image_by_id, fps=1)

seq_dfs = get_sequences(df, interval=60)  # break the data by intervals between sequences
print(df.shape, len(seq_dfs))

# for i,sub_df in enumerate(seq_dfs):
#     ids = sub_df.id.to_list()
#     video_name = os.path.join(data_dir, f'dust_seq{i}.mp4')
#     create_video(ids, data_dir, video_name, read_raw_image_by_id, fps=3)

(39034, 106) 13


### select images for labeling

In [31]:
# combine and create datasets for human in dust
data_root_dir = '/data/jupiter/datasets'
unlabeled_datasets = [
    "halo_human_in_dust_day_collection_may29",
    "halo_human_in_dust_night_collection_june03",  # for lying down humans
    "halo_human_in_dust_night_collection_june03_2",
    "halo_human_in_dust_day_collection_back_june05",
    "halo_human_in_dust_dusk_collection_front_june07",
]
day_night_splits = ['day', 'dawn_dusk', {'dawn_dusk': list(range(0, 9)), 'night': list(range(9, 19))}, 'day', 'dawn_dusk']
use_ds = [0, 2, 3, 4]
# split data by day time
image_ids_by_time = {'id':[], 'day_night_split': []}
for di in use_ds:
    dataset = unlabeled_datasets[di]
    df = pd.read_csv(os.path.join(data_root_dir, dataset, 'annotations.csv'))
    df = df.sort_values('collected_on')
    day_night_split = day_night_splits[di]
    if isinstance(day_night_split, str):
        print(dataset, df.shape, day_night_split)
        image_ids_by_time['id'] += df.id.to_list()
        image_ids_by_time['day_night_split'] += [day_night_split] * len(df)
    else:
        seq_dfs = get_sequences(df, interval=60)  # break the data by intervals between sequences
        print(dataset, df.shape, len(seq_dfs), day_night_split)
        for dn_split, seq_ids in day_night_split.items():
            for seq_id in seq_ids:
                seq_df = seq_dfs[seq_id]
                image_ids_by_time['id'] += seq_df.id.to_list()
                image_ids_by_time['day_night_split'] += [dn_split] * len(seq_df)
# save to csv
day_night_split_df = pd.DataFrame(data=image_ids_by_time)
print(day_night_split_df.shape)
day_night_split_df.to_csv('/data/jupiter/li.yu/data/halo_hard_cases/halo_human_in_dust_daynightsplit_for_july01.csv')

halo_human_in_dust_day_collection_may29 (48102, 108) day
halo_human_in_dust_night_collection_june03_2 (30284, 109) 19 {'dawn_dusk': [0, 1, 2, 3, 4, 5, 6, 7, 8], 'night': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]}
halo_human_in_dust_day_collection_back_june05 (11675, 108) day
halo_human_in_dust_dusk_collection_front_june07 (36338, 104) dawn_dusk
(126399, 2)


In [27]:
# image_ids_by_time = {'id':[], 'day_night_split': []}
# split data in day/dusk/night
day_night_splits = [{'day': list(range(0,25)), 'dawn_dusk': list(range(25, 31)), 'night': list(range(31, 38))}, 
                   {'dawn_dusk': list(range(0, 14)), 'night': list(range(14, 29))},
                   {'dawn_dusk': [0, 1, 2, 7, 8, 9], 'night': [3, 4, 5, 6, 10, 11, 12]}]
day_night_split = day_night_splits[2]
for dn_split, seq_ids in day_night_split.items():
    for seq_id in seq_ids:
        seq_df = seq_dfs[seq_id]
        image_ids_by_time['id'] += seq_df.id.to_list()
        image_ids_by_time['day_night_split'] += [dn_split] * len(seq_df)
# save to csv
day_night_split_df = pd.DataFrame(data=image_ids_by_time)
print(day_night_split_df.shape)
day_night_split_df.to_csv('/data/jupiter/li.yu/data/halo_hard_cases/halo_vehicle_in_dust_daynightsplit_excludebadnight_for_july01.csv')

(176217, 2)


In [19]:
day_night_split_df.head(2)

,id,day_night_split
0,65e9e939e879f487a282d576,day
1,65e9e9371d130730de6a5943,day


In [7]:
# # for selection of images for binary labeling - halo_failure_case_of_box_in_dust
# selected = [3, 5, [9, 1/5], [10, 1/5], [11, 1/6], 
#     [12, [['2024-04-23T18:11:28', '2024-04-23T18:18:12'], ['2024-04-23T18:27:40', '2024-04-23T18:28:38'], ['2024-04-23T18:29:05', '2024-04-23T18:29:57'], 
#         ['2024-04-23T18:48:44', '2024-04-23T18:50:59'], ['2024-04-23T19:01:50', '2024-04-23T19:03:37']]], 
#     [13, [['2024-04-23T19:47:48', '2024-04-23T19:47:59']]], 
#     [14, [['2024-04-23T20:12:40', '2024-04-23T20:13:02'], ['2024-04-23T20:15:53', '2024-04-23T20:17:49'], ['2024-04-23T20:22:19', '2024-04-23T20:22:45'], 
#         ['2024-04-23T20:32:47', '2024-04-23T20:33:18'], ['2024-04-23T20:34:48', '2024-04-23T20:35:56'], ['2024-04-23T20:38:00', '2024-04-23T20:38:17'], 
#         ['2024-04-23T20:39:55', '2024-04-23T20:41:31'], ['2024-04-23T20:42:57', '2024-04-23T20:48:27']]], 
#     [16, 1/2]]
# selected_ids = []
# for s in selected:
#     if isinstance(s, int):
#         selected_ids += seq_dfs[s].id.to_list()
#     else:
#         si, sj = s
#         if isinstance(sj, float):
#             selected_ids += seq_dfs[si].iloc[:int(len(seq_dfs[si])*sj)].id.to_list()
#         else:
#             for t1, t2 in sj:
#                 selected_ids += seq_dfs[si][(seq_dfs[si].collected_on >= t1) & (seq_dfs[si].collected_on <= t2)].id.to_list()
#     # print(s, len(selected_ids))
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_failure_case_of_box_in_dust/selected_for_label.csv', index=False)

In [8]:
def recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence):
    all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}

    raw_ms_df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))
    raw_ms_df['camera_pair'] = raw_ms_df['unique_id'].apply(lambda s: s[-7:])
    raw_ms_df['rectified_label_save_path'] = ''
    raw_ms_df['use_recovered_label_in_sequence'] = True
    labeled_ms_df = pd.read_csv(os.path.join(data_dir+suffix, 'master_annotations.csv'))
    labeled_ms_df.drop(columns=["label_counts"], inplace=True)
    labeled_ms_df['camera_pair'] = labeled_ms_df['unique_id'].apply(lambda s: s[-7:])
    print(raw_ms_df.shape, labeled_ms_df.shape)

    recovered_dfs = []
    for pod, seq_ids in same_human_sequence.items():
        for seq_id in seq_ids:
            # get seq df in pod
            seq_df = seq_dfs[seq_id]
            seq_df = seq_df[seq_df.camera_location.isin(all_cameras[pod])]
            # get corresponding raw seq df and labeled seq df
            raw_seq_df = raw_ms_df[raw_ms_df.id.isin(seq_df.id)]
            labeled_seq_df = labeled_ms_df[labeled_ms_df.id.isin(seq_df.id)]
            labeled_seq_df = labeled_seq_df.sort_values('collected_on')
            # get camera locations where there are human labels
            labeled_camera_pairs = labeled_seq_df.camera_pair.unique()
            for camera_pair in labeled_camera_pairs:
                raw_seq_cp_df = raw_seq_df[raw_seq_df.camera_pair == camera_pair]
                labeled_seq_cp_df = labeled_seq_df[labeled_seq_df.camera_pair == camera_pair]
                # assign label path to raw df
                for i, row in raw_seq_cp_df.iterrows():
                    labeled_rows = labeled_seq_cp_df[labeled_seq_cp_df.unique_id == row.unique_id]
                    if len(labeled_rows) > 0:
                        raw_seq_cp_df.loc[i, 'rectified_label_save_path'] = labeled_rows.iloc[0].rectified_label_save_path
                        raw_seq_cp_df.loc[i, 'use_recovered_label_in_sequence'] = False
                    else:
                        raw_seq_cp_df.loc[i, 'rectified_label_save_path'] = labeled_seq_cp_df.iloc[0].rectified_label_save_path
                recovered_dfs.append(raw_seq_cp_df)
            print(pod, seq_id, len(seq_df), len(raw_seq_df), len(labeled_seq_df), raw_seq_df.camera_pair.unique(), labeled_seq_df.camera_pair.unique())
    recovered_df = pd.concat(recovered_dfs, ignore_index=True)
    print(recovered_df.shape, labeled_ms_df.shape)

    # remove duplicates and add in extra sequence images from labeled_ms_df
    recovered_df = pd.concat([recovered_df.drop_duplicates(subset='unique_id'), labeled_ms_df[~labeled_ms_df.unique_id.isin(recovered_df.unique_id)]], ignore_index=True)
    # change relative path to label path
    recovered_df['rectified_label_save_path'] = recovered_df['rectified_label_save_path'].apply(lambda p: f'../{os.path.basename(data_dir)+suffix}/{p}')
    recovered_df['label_map'] = labeled_ms_df.iloc[0].label_map
    print(recovered_df.shape, labeled_ms_df.shape)
    recovered_df.to_csv(os.path.join(data_dir, 'label_recovered_master_annotations.csv'), index=False)

In [9]:
# # for selection of images for partial human labeling - halo_human_in_dust_day_collection_may29
# selected = {'front': {0: 'all', 1: ['begin', '2024-05-29T19:48:13'], 2: ['2024-05-29T20:28:50', 'end']},
#             'right': {2: ['begin', '2024-05-29T20:28:53']},
#             'back': {1: ['2024-05-29T20:02:08', 'end']},
#             'left': {1: ['2024-05-29T19:48:10', '2024-05-29T20:02:08']}}
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# left_cameras = ['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left', 'T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02']

# selected_ids = []
# for pod, cameras in all_cameras.items():
#     cameras = set(cameras).intersection(left_cameras)
#     selected_times = selected[pod]
#     for seq_i, t1t2 in selected_times.items():
#         seq_df = seq_dfs[seq_i]
#         seq_df = seq_df[seq_df.camera_location.isin(cameras)]
#         if t1t2 == 'all':
#             t1, t2 = seq_df.iloc[0].collected_on, seq_df.iloc[-1].collected_on
#         else:
#             t1, t2 = t1t2
#             if t1 == 'begin':
#                 t1 = seq_df.iloc[0].collected_on
#             if t2 == 'end':
#                 t2 = seq_df.iloc[-1].collected_on
#         print(pod, seq_i, t1, t2, len(seq_df))
#         selected_ids += seq_df[(seq_df.collected_on >= t1) & (seq_df.collected_on <= t2)].id.to_list()
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_human_in_dust_day_collection_may29/selected_for_label.csv', index=False)

# recover skipped human labels in heavy dust, by images in the same sequence
same_human_sequence = {'front': [0, 1, 3, 14], 'right': [11, 12, 13], 'back': [7, 8, 9, 10], 'left': [4, 5, 6]}
suffix = '_human_labeled_stereo'
recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)

<ipython-input-9-56d6a59a9e9d>:34: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)
/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


(35854, 153) (5967, 275)
front 0 599 446 246 ['T01_T03' 'T02_T03' 'T02_T04'] ['T02_T04' 'T02_T03']
front 1 219 164 74 ['T01_T03' 'T02_T03' 'T02_T04'] ['T02_T04' 'T02_T03']
front 3 145 108 42 ['T01_T03' 'T02_T03' 'T02_T04'] ['T02_T04' 'T02_T03']
front 14 1156 867 284 ['T01_T03' 'T02_T04' 'T02_T03'] ['T01_T03' 'T02_T03' 'T02_T04']
right 11 173 120 117 ['T05_T07' 'T06_T07' 'T06_T08'] ['T06_T08' 'T06_T07' 'T05_T07']
right 12 951 709 705 ['T05_T07' 'T06_T07' 'T06_T08'] ['T06_T08' 'T05_T07' 'T06_T07']
right 13 1095 820 535 ['T05_T07' 'T06_T07' 'T06_T08'] ['T06_T07' 'T06_T08']
back 7 1274 948 943 ['T09_T11' 'T10_T11' 'T10_T12'] ['T10_T11' 'T09_T11' 'T10_T12']
back 8 1146 854 558 ['T09_T11' 'T10_T11' 'T10_T12'] ['T10_T12' 'T10_T11']
back 9 454 332 222 ['T09_T11' 'T10_T11' 'T10_T12'] ['T10_T11' 'T10_T12']
back 10 897 670 447 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T12' 'T10_T11']
left 4 1177 875 805 ['T13_T15' 'T14_T16' 'T14_T15'] ['T14_T16' 'T14_T15' 'T13_T15']
left 5 1223 912 502 ['T13_T15' 'T1

In [18]:
# # load in labeled dataset
# labeled_csv = '/data2/jupiter/datasets/halo_vehicles_driving_through_dust_images_nodust_reserved_labeled_maxfov_alleysson_depth0423/annotations.csv'
# labeled_df = pd.read_csv(labeled_csv)
# print(labeled_df.shape)
# labeled_ids = set(labeled_df.id.to_list())
# print(len(labeled_ids))

# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_march2024
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [2, 3, 4, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 30, 31, 32, 33, 34, 35, 36], 
#             'right_pass': [3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 21, 26, 27, 37]}
# selected_ids = []
# for pod, cameras in left_cameras.items():
#     print(pod, cameras)
#     for i,seq_df in enumerate(seq_dfs):
#         if i in selected[pod]:
#             sub_df = seq_df[seq_df.camera_location.isin(cameras)]
#             selected_ids += sub_df.id.to_list()
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df = selected_df[~selected_df.id.isin(labeled_ids)]
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_vehicles_in_dust_collection_march2024/selected_for_missing_label.csv', index=False)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(13141, 1)
(9893, 1)


In [69]:
# # for selection of images for missing partial human labeling - halo_human_in_dust_night_collection_june03
# left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# selected = {'front': [25, 26, 27, 30]}
# selected_ids = []
# for pod, cameras in all_cameras.items():
#     print(pod, cameras)
#     for i,seq_df in enumerate(seq_dfs):
#         if i in selected[pod]:
#             sub_df = seq_df[seq_df.camera_location.isin(cameras)]
#             selected_ids += sub_df.id.to_list()

# # for selection of images for missing partial human labeling - halo_human_in_dust_night_collection_june03_2
# left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [0, 1, 2, 3, 15, 16, 17], 'right': [18,], 'back': [7, 8, 9, 10, 11], 'left': [4, 5, 6, 12, 13, 14]}
# selected_ids = []
# for pod, cameras in all_cameras.items():
#     print(pod, cameras)
#     for i,seq_df in enumerate(seq_dfs):
#         if i in selected[pod]:
#             sub_df = seq_df[seq_df.camera_location.isin(cameras)]
#             selected_ids += sub_df.id.to_list()
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_human_in_dust_night_collection_june03_2/selected_for_label.csv', index=False)

# recover skipped human labels in heavy dust, by images in the same sequence
same_human_sequence = {'front': [15, 17], 'right': [18], 'back': [7, 8, 9, 10], 'left': [5, 6]}
suffix = '_human_labeled_stereo'
recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)

(22675, 153) (11839, 278)
front 15 1471 1101 691 ['T01_T03' 'T02_T03' 'T02_T04'] ['T02_T04' 'T02_T03']
front 17 809 606 281 ['T01_T03' 'T02_T03' 'T02_T04'] ['T02_T04' 'T01_T03' 'T02_T03']
right 18 748 559 361 ['T05_T07' 'T06_T07' 'T06_T08'] ['T06_T08' 'T06_T07']
back 7 932 697 161 ['T09_T11' 'T10_T11' 'T10_T12'] ['T09_T11']
back 8 564 423 216 ['T09_T11' 'T10_T11' 'T10_T12'] ['T10_T12' 'T10_T11']
back 9 1368 1025 583 ['T09_T11' 'T10_T11' 'T10_T12'] ['T10_T12' 'T10_T11']
back 10 1552 1161 809 ['T09_T11' 'T10_T11' 'T10_T12'] ['T10_T11' 'T09_T11' 'T10_T12']
left 5 1104 826 488 ['T13_T15' 'T14_T15' 'T14_T16'] ['T14_T15' 'T14_T16']
left 6 1460 1090 676 ['T13_T15' 'T14_T15' 'T14_T16'] ['T14_T16' 'T14_T15']
(5346, 153) (11839, 278)
(12919, 279) (11839, 278)


In [64]:
# for selection of images for missing partial human labeling - halo_human_in_dust_day_collection_back_june05
# recover skipped human labels in heavy dust, by images in the same sequence
same_human_sequence = {'back': [0, 1, 2, 3, 5, 6, 7]}
suffix = '_human_labeled_stereo'
recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)

<ipython-input-64-9e5b735fb503>:5: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  recovered_df, labeled_ms_df = recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)
/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


(8740, 153) (3511, 277)
back 0 752 562 184 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 1 1216 911 114 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 2 248 186 11 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 3 1476 1102 196 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 5 1688 1262 576 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T12' 'T10_T11']
back 6 2176 1631 967 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T11' 'T10_T12']
back 7 1380 1034 687 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T12' 'T10_T11']
(3538, 153) (3511, 277)
(4314, 278) (3511, 277)


In [8]:
# # for selection of images for missing partial human labeling - halo_human_in_dust_dusk_collection_front_june07
# left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [5], 'right': [], 'back': [], 'left': []}
# selected_ids = []
# for pod, cameras in left_cameras.items():
#     print(pod, cameras)
#     for i,seq_df in enumerate(seq_dfs):
#         if i in selected[pod]:
#             sub_df = seq_df[seq_df.camera_location.isin(cameras)]
#             selected_ids += sub_df.id.to_list()
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_human_in_dust_dusk_collection_front_june07/selected_for_label.csv', index=False)

front ['T01', 'T02']
right ['T05', 'T06']
back ['T09', 'T10']
left ['T13', 'T14']
(3354, 1)


In [14]:
# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_june06
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [11, 12, 15], 
#             'right_pass': [10, 13, 14]}
# selected_ids = []
# for pod, cameras in left_cameras.items():
#     print(pod, cameras)
#     for i,seq_df in enumerate(seq_dfs):
#         if i in selected[pod]:
#             sub_df = seq_df[seq_df.camera_location.isin(cameras)]
#             selected_ids += sub_df.id.to_list()
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_vehicles_in_dust_collection_june06/selected_for_label.csv', index=False)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(4513, 1)


In [8]:
# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_june07
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [2, 6, 7, 10], 
#             'right_pass': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
# selected_ids = []
# for pod, cameras in left_cameras.items():
#     print(pod, cameras)
#     for i,seq_df in enumerate(seq_dfs):
#         if i in selected[pod]:
#             sub_df = seq_df[seq_df.camera_location.isin(cameras)]
#             selected_ids += sub_df.id.to_list()
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_vehicles_in_dust_collection_june07/selected_for_label.csv', index=False)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(14354, 1)


### create video

In [6]:
def read_csvs(pred_root_dir, model, labeled_dataset):
    pred_df = pd.read_csv(os.path.join(pred_root_dir, model, labeled_dataset, 'output.csv'))
    if not 'state' in pred_df:
        pred_df['state'] = pred_df['result_state']
    dust_df = pd.read_csv(os.path.join(pred_root_dir, model, labeled_dataset, 'dust_ratio.csv'))
    print(labeled_dataset, pred_df.shape, dust_df.shape)
    df = pred_df[['unique_id', 'state', 'camera_location', 'operation_time']].merge(dust_df, on='unique_id')
    all_cameras = {'Front Pod': ['T01', 'T02', 'T03', 'T04'], 'Right Pod': ['T05', 'T06', 'T07', 'T08'], 'Rear Pod': ['T09', 'T10', 'T11', 'T12'], 'Left Pod': ['T13', 'T14', 'T15', 'T16']}
    all_cameras = {c: pod for pod, cameras in all_cameras.items() for c in cameras}
    df['pod'] = df.camera_location.apply(lambda c: all_cameras[c])
    return df

In [7]:
# load in model predictions
pred_root_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
model = 'all_rev2_rev1_lying_down_sitting_headlights_round_2_25_ep_prod_weights_10_lo_10_tr_dust_0601'
suffix = '_human_labeled_stereo'
pred_df = read_csvs(pred_root_dir, model, os.path.basename(data_dir)+suffix)
print(pred_df.shape)
pred_df.head(2)

halo_human_in_dust_day_collection_may29_human_labeled_stereo (5968, 16) (5968, 7)
(5968, 11)


,unique_id,state,camera_location,operation_time,id,gt_dust_ratio,total_averaged_dust_conf,total_thresholded_dust_ratio,masked_avg_dust_conf,masked_dust_ratio,pod
0,66595e558b545d525752cfb2_T02_T03,true_positive,T02,daytime,66595e558b545d525752cfb2,0.0,0.124939,0.036865,0.152748,0.047904,Front Pod
1,66595eea568146714ce37514_T02_T03,true_positive,T02,daytime,66595eea568146714ce37514,0.0,0.125615,0.028809,0.125125,0.029940,Front Pod


In [8]:
seq_df = seq_dfs[0]
pred_seq_df = pred_df[pred_df.id.isin(seq_df.id)]
print(seq_df.shape, pred_seq_df.shape)
pred_seq_df.groupby('state').size()

(1330, 109) (246, 11)


state
false_negative      1
false_positive      5
true_negative     126
true_positive     114
dtype: int64

In [33]:
# read from multiple cameras and put in once frame
# left_pass_pairs = ['T09_T11', 'T14_T16', 'T14_T15', 'T13_T15']
# right_pass_pairs = ['T05_T07', 'T10_T12', 'T06_T08', 'T06_T07']
# cameras = [f'T{str(i+1).zfill(2)}' for i in range(16)][12:]
# cameras = ['T16', 'T15', 'T14', 'T13', 'T11', 'T09']
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# all_left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# left_cameras = ['T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14']
# back_left_cameras = ['T09', 'T10']
left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
for pod, cameras in left_cameras.items():
    # pod = 'all_left'
    # cameras = left_cameras
    print(pod, cameras)
    H = 2  # number of camera rows
    W = 2  # number of camera cols
    for i,seq_df in enumerate(seq_dfs):
        # # check model prediction and filter by dust/seg outputs
        # pred_seq_df = pred_df[pred_df.id.isin(seq_df.id)]
        # labeled_ids = pred_seq_df.id.to_list()

        cam_dfs = [seq_df[seq_df.camera_location == c] for c in cameras]
        print(i, [len(cdf) for cdf in cam_dfs])
        min_len = min(len(cdf) for cdf in cam_dfs)
        cam_dfs = [cdf.sort_values('collected_on').iloc[:min_len] for cdf in cam_dfs]
        if min_len < 2:
            continue
        # print(i, [cdf.iloc[0].collected_on for cdf in cam_dfs])

        frame = read_raw_image_by_row(data_dir, seq_df.iloc[0])
        height, width, layers = frame.shape
        # print(height, width, layers)

        # .avi MJPG,  .mp4 MP4V
        video_dir = os.path.join(data_dir, f'videos_{len(cameras)}cams')
        os.makedirs(video_dir, exist_ok=True)
        video_name = os.path.join(video_dir, f'{pod}_seq{str(i).zfill(2)}.mp4')
        video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 3, (width*W,height*H), isColor=True)

        for ii in tqdm(range(min_len)):
            try:
                canvas = np.zeros((height*H, width*W, 3), dtype=np.uint8)
                for ci in range(len(cam_dfs)):
                    cam_df_row = cam_dfs[ci].iloc[ii]
                    frame = read_raw_image_by_row(data_dir, cam_df_row)
                    s = f'{cam_df_row.camera_location} {cam_df_row.collected_on}'
                    # if cam_df_row.id in labeled_ids:
                    #     pred_row = pred_seq_df[pred_seq_df.id == cam_df_row.id].iloc[0]
                    #     s += f' pred: {pred_row.state}, dust: {pred_row.total_averaged_dust_conf:.4f}'
                    frame = cv2.putText(frame, s, 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
                    fi, fj = ci // W, ci % W
                    canvas[height*fi:height*(fi+1), width*fj:width*(fj+1)] = frame
                video.write(canvas)
            except:
                print(f'{ii}th image read failed')

        # cv2.destroyAllWindows()
        video.release()
        # break

front ['T01', 'T02', 'T03', 'T04']
0 [436, 436, 436, 437]
1 [588, 589, 589, 589]
2 [1130, 1131, 1130, 1131]
3 [692, 692, 692, 692]
4 [0, 0, 0, 0]
5 [0, 0, 0, 0]
6 [0, 0, 0, 0]
7 [0, 0, 0, 0]
8 [0, 0, 0, 0]
9 [0, 0, 0, 0]
10 [0, 0, 0, 0]
11 [0, 0, 0, 0]
12 [0, 0, 0, 0]
13 [0, 0, 0, 0]
14 [0, 0, 0, 0]
15 [368, 367, 368, 368]
16 [337, 337, 337, 337]
17 [202, 203, 203, 201]
18 [0, 0, 0, 0]
right ['T05', 'T06', 'T07', 'T08']
0 [0, 0, 0, 0]
1 [0, 0, 0, 0]
2 [0, 0, 0, 0]
3 [0, 0, 0, 0]
4 [0, 0, 0, 0]
5 [0, 0, 0, 0]
6 [0, 0, 0, 0]
7 [0, 0, 0, 0]
8 [0, 0, 0, 0]
9 [0, 0, 0, 0]
10 [0, 0, 0, 0]
11 [0, 0, 0, 0]
12 [0, 0, 0, 0]
13 [0, 0, 0, 0]
14 [0, 0, 0, 0]
15 [0, 0, 0, 0]
16 [0, 0, 0, 0]
17 [0, 0, 0, 0]
18 [187, 187, 187, 187]
back ['T09', 'T10', 'T11', 'T12']
0 [0, 0, 0, 0]
1 [0, 0, 0, 0]
2 [0, 0, 0, 0]
3 [0, 0, 0, 0]
4 [0, 0, 0, 0]
5 [0, 0, 0, 0]
6 [0, 0, 0, 0]
7 [233, 233, 233, 233]
8 [141, 141, 141, 141]
9 [342, 342, 342, 342]
10 [388, 388, 388, 388]
11 [384, 384, 384, 384]
12 [0, 0, 0, 0]
13

## Create video from saved pngs

In [ ]:
def create_video(ids, pred_dir, video_name, read_func=read_saved_frame, fps=2):
    frame = read_func(pred_dir, ids[10])
    height, width, layers = frame.shape
    print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width,height), isColor=True)
    
    good = 0
    for _id in tqdm(ids):
        frame = read_func(pred_dir, _id)
        if frame is not None:
            video.write(frame)
            good += 1
    print('total', len(ids), 'used', good)

    cv2.destroyAllWindows()
    video.release()

In [10]:
data_dir = '/data/jupiter/li.yu/data/Jupiter_human_on_path_3_fn_sequence'
# csv_file = '/data/jupiter/datasets/Jupiter_gilroy_reverse_manny_v2/master_annotations_1fps.csv'
# csv_file = '/data/jupiter/li.yu/data/mannequin_in_dust_v1/master_annotations.csv'
csv_file = f'{data_dir}/master_annotations.csv'
master_df = pd.read_csv(csv_file)
master_df = master_df.sort_values('collected_on')

# pred_dir = '/mnt/sandbox1/rakhil.immidisetti/output/driveable_terrain_model/471_cloud_v45_cutnpaste_s35/Jupiter_gilroy_reverse_manny_v2reverse_val_bestmodel/'
# pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/v471_nobadiqnohard_6cls_cnp_alpha2_0805/Jupiter_gilroy_reverse_manny_v2/'
# pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/mannequin_in_dust_v1'
pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/Jupiter_human_on_path_3_fn_sequence'
save_dir = pred_dir

# # save as a single video
# print(master_df.shape)
# video_name = os.path.join(save_dir, 'pred.mp4')
# ids = master_df.image_id.to_list()
# create_video(ids, pred_dir, video_name, fps=3)

# break into sequences
seq_dfs = get_sequences(master_df, interval=5, per_camera=True)
print(len(master_df), len(seq_dfs))
video_dir = os.path.join(pred_dir, 'videos')
os.makedirs(video_dir, exist_ok=True)
for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # create video
    # video_name = os.path.join(video_dir, str(si).zfill(3)+'.mp4')
    video_name = os.path.join(video_dir, f'{name}_{camera}_{si}.mp4')
    ids = seq_df.image_id.to_list()
    create_video(ids, pred_dir, video_name, fps=3)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


208 3
0 2021-11-10T00:33:05.931000 49
311 3141 3


100%|██████████| 49/49 [00:01<00:00, 37.81it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


total 49 used 49
1 2021-12-16T22:14:00.050000 71
311 3141 3


100%|██████████| 71/71 [00:01<00:00, 38.55it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


total 71 used 71
2 2022-11-10T23:19:10.901000 88
311 3141 3


100%|██████████| 88/88 [00:02<00:00, 40.61it/s]

total 88 used 88


### Create video from PP artifacts

In [16]:
def read_from_pp_artifacts(data_dir, df_row):
    data_path = os.path.join(data_dir, df_row.stereo_pipeline_npz_save_path)
    img = np.load(data_path)['left']
    img_norm = normalize_image(img, df_row.hdr_mode if 'hdr_mode' in df_row else True)
    return cv2.cvtColor((img_norm * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)

# def add_text(frame, txt_row):
#     frame = cv2.putText(frame, f'Collected on: {txt_row.collected_on}', 
#                         (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
#     return frame

def add_text(frame, txt):
    frame = cv2.putText(frame, txt, 
                        (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    return frame

def add_texts(frame, txts: list):
    txt_pw, txt_ph = 10, 25
    for i, txt in enumerate(txts):
        if 'false_positive' in txt:
            frame = cv2.putText(frame, txt, 
                                (txt_pw, txt_ph+i*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
        else:
            frame = cv2.putText(frame, txt, 
                                (txt_pw, txt_ph+i*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    return frame

In [5]:
# data_root_dir = '/data/jupiter/li.yu/data'
data_root_dir = '/data/jupiter/datasets/'
# dataset = 'mannequin_in_dust_v1'
# dataset = 'Jupiter_human_on_path_3_fn_sequence'
dataset = 'halo_missed_lo_rock_0509_stereo'
data_dir = os.path.join(data_root_dir, dataset)

df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))
seq_dfs = get_sequences(df, interval=5, per_camera=True)
print(df.shape, len(seq_dfs))

/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (37,46,58,60,74,86,101,103,106,116,118) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(23023, 150) 432


In [6]:
bags = [
["05_09_2024_18_16_02", "05_09_2024_18_17_02"],
["05_09_2024_18_17_01", "05_09_2024_18_18_01"],
["05_09_2024_18_17_48", "05_09_2024_18_18_48"],
["05_09_2024_18_18_45", "05_09_2024_18_19_45"],
["05_09_2024_18_21_31", "05_09_2024_18_22_31"],
["05_09_2024_18_22_26", "05_09_2024_18_23_26"],
["05_09_2024_18_23_12", "05_09_2024_18_24_12"],
["05_09_2024_18_24_15", "05_09_2024_18_25_15"],
["05_09_2024_20_16_20", "05_09_2024_20_17_20"],
["05_09_2024_20_18_38", "05_09_2024_20_19_38"],
["05_09_2024_20_20_28", "05_09_2024_20_21_28"],
]
bags = [[bag[0][11:].replace('_', ':'), bag[1][11:].replace('_', ':')] for bag in bags]
df['collected_hms'] = df['collected_on'].apply(lambda t: t[11:])
df_oi = []
for bag in bags:
    sub_df = df[(df.collected_hms >= bag[0]) & (df.collected_hms < bag[1])]
    print(bag, len(sub_df))
    df_oi.append(sub_df)
df_oi = pd.concat(df_oi, ignore_index=True).drop_duplicates()
print(df_oi.shape)

seq_dfs = get_sequences(df_oi)
print(df_oi.shape, len(seq_dfs))

['18:16:02', '18:17:02'] 606
['18:17:01', '18:18:01'] 750
['18:17:48', '18:18:48'] 498
['18:18:45', '18:19:45'] 354
['18:21:31', '18:22:31'] 732
['18:22:26', '18:23:26'] 900
['18:23:12', '18:24:12'] 426
['18:24:15', '18:25:15'] 432
['20:16:20', '20:17:20'] 930
['20:18:38', '20:19:38'] 850
['20:20:28', '20:21:28'] 1200
(6568, 151)
(6568, 151) 2


In [8]:
pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model'
models = [
    'ds_v8_1_nextvit_small_openimages_with_rev1_train_human_test_using_random_val_mhc_20_epochs_finetune_rev1_lr',
    'v81_80k_maxfov_wn_ft_kore_0430'
]
suffix_list = ['_mhc_depth0125', '']
pred_dfs = [pd.read_csv(os.path.join(pred_dir, model, dataset+suffix, 'output.csv')) for model,suffix in zip(models, suffix_list)]
pred_dfs[0].head(1)

,unique_id,id,camera_location,operation_time,special_notes,jdb_s3_path,result_state,result_human_state,result_vehicle_state,min_pixels_threshold,features,n_gt_human_pixels,gt_human_depth,n_pred_human_pixels,pred_human_depth
0,663ec898321f043ed7ad8a62_T02_T03,663ec898321f043ed7ad8a62,T02,daytime,NaN,NaN,true_negative,true_negative,true_negative,108,"{""large_object_pixels"": 688, ""large_object_min...",0,1000,0,1000


In [18]:
def get_pred_texts(pred_dfs, unique_id, text_prefix):
    texts = []
    for pred_df, prefix in zip(pred_dfs, text_prefix):
        pred_row = pred_df[pred_df.unique_id == unique_id].iloc[0]
        texts.append(f'{prefix}: {pred_row.result_state}')
    return texts

def create_video_from_pp_add_text(seq_df, camera_pairs, pred_dfs, H, W, data_dir, video_dir_name):
    cam_dfs = [seq_df[seq_df.unique_id.str.endswith(c)] for c in camera_pairs]
    cam_dfs = [cdf.sort_values('collected_on', ignore_index=True) for cdf in cam_dfs]
    min_len = min(len(cdf) for cdf in cam_dfs)
    cam_dfs = [cdf.iloc[:min_len] for cdf in cam_dfs]
    print([len(cdf) for cdf in cam_dfs])

    # .avi MJPG,  .mp4 MP4V
    os.makedirs(os.path.join(data_dir, video_dir_name), exist_ok=True)
    video_name = os.path.join(data_dir, f'{video_dir_name}/{seq_df.iloc[0].collected_on}.mp4')

    # print(f'{H} rows, {W} cols out of {len(camera_pairs)} camera_pairs')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 3, (768*W,512*H), isColor=True)

    for ii in tqdm(range(min_len)):
        # try:
        canvas = np.zeros((512*H, 768*W, 3), dtype=np.uint8)
        for ci in range(len(cam_dfs)):
            row = cam_dfs[ci].iloc[ii]
            frame = read_from_pp_artifacts(data_dir, row)
            texts = [f'{row.collected_on}'] + get_pred_texts(pred_dfs, row.unique_id, ['MHC pred', 'MAXFOV pred'])
            # print(texts)
            frame = add_texts(frame, texts)
            fi, fj = ci // W, ci % W
            canvas[512*fi:512*(fi+1), 768*fj:768*fj+frame.shape[1]] = frame
        video.write(canvas)
        # except:
        #     print(f'{ii}th image read failed')

    # cv2.destroyAllWindows()
    video.release()

front_pairs = ['T02_T04', 'T02_T03', 'T01_T03']
left_pass_pairs = ['T02_T04', 'T02_T03', 'T01_T03', 'T13_T15', 'T14_T15', 'T14_T16']
right_pass_pairs = ['T02_T04', 'T02_T03', 'T01_T03', 'T06_T08', 'T05_T07', 'T06_T07']
# left_pass_pairs = ['T09_T11', 'T14_T16', 'T14_T15', 'T13_T15']
# right_pass_pairs = ['T05_T07', 'T10_T12', 'T06_T08', 'T06_T07']
H = 1  # number of image rows
W = 3  # number of image cols
create_video_from_pp_add_text(seq_dfs[1], front_pairs, pred_dfs, H, W, data_dir, 'videos_front_pass')

  0%|          | 1/248 [00:00<00:29,  8.30it/s]

[248, 248, 248]


100%|██████████| 248/248 [00:30<00:00,  8.21it/s]


In [36]:
df["camera_pair"] = df["unique_id"].apply(lambda t: t[-7:])
df["camera_pair"].unique()

array(['T01_T03', 'T02_T03', 'T02_T04', 'T05_T07', 'T06_T07', 'T06_T08',
       'T09_T11', 'T10_T12', 'T10_T11', 'T13_T15', 'T14_T16', 'T14_T15'],
      dtype=object)

### Read frame and add prediction results

In [20]:
# compare BRT model pred and CenterTrack pred
# pred_csv = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/mannequin_in_dust_v1/output.csv'
pred_csv = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/Jupiter_human_on_path_3_fn_sequence/output.csv'
pred_df = pd.read_csv(pred_csv)
print(pred_df.shape)

(208, 11)


In [11]:
video_with_pred_dir = os.path.join(data_root_dir, dataset, 'videos_with_pred')
# pred_sequence_dir = '/home/li.yu/code/CenterTrack/results/2023-07-08T01:37:09.798000_front-center_15'
pred_sequence_dir = '/home/li.yu/code/CenterTrack/results/brt50000/nopreimg_noprehm/2023-07-08T01:37:09.798000_front-center_15'
os.makedirs(video_with_pred_dir, exist_ok=True)
height, width = 512, 1024

for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10 or si != 15:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # merge pred from BRT model
    seq_df = seq_df.drop(columns=['state']).merge(pred_df[['id', 'state', 'human_state']], on='id')

    # create video
    video_name = os.path.join(video_with_pred_dir, f'{name}_{camera}_{si}_nopreimg_noprehm.mp4')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 3, (width,height), isColor=True)
    fi = 0
    for _, df_row in seq_df.iterrows():
        frame = cv2.imread(os.path.join(pred_sequence_dir, str(fi).zfill(3)+'_'+df_row.id+'.png'))
        frame = cv2.putText(frame, f'BRT Seg Pred: {df_row.state}, Strict: {df_row.human_state}', 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        video.write(frame)
        fi += 1
    cv2.destroyAllWindows()
    video.release()

    # break


15 2023-07-08T01:37:09.798000 153


### read from video and prediction results to each frame

In [23]:
video_dir = '/home/li.yu/code/CenterTrack/results/'
old_video_name = 'brt50000_2022-11-10T23:19:10.901000_side-right_2_preimg.mp4'
new_video_name = 'brt50000_2022-11-10T23:19:10.901000_side-right_2_preimg_withbrtpred.mp4'
height, width = 512, 1024

for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10 or si != 2:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # merge pred from BRT model
    seq_df = seq_df.drop(columns=['state']).merge(pred_df[['id', 'state', 'human_state']], on='id')

    # read video
    cam = cv2.VideoCapture(os.path.join(video_dir, old_video_name))

    # create video
    video_name = os.path.join(video_dir, new_video_name)
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 3, (width,height), isColor=True)
    for _, df_row in seq_df.iterrows():
        _, frame = cam.read()
        frame = cv2.putText(frame, f'BRT Seg Pred: {df_row.state}, Strict: {df_row.human_state}', 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        video.write(frame)
    cv2.destroyAllWindows()
    video.release()

    # break


2 2022-11-10T23:19:10.901000 88
